In [ ]:
import spacy
from spacy import displacy

nlp = spacy.load("en_core_web_md") 
text = "The quick brown fox jumps over the lazy dog."
doc = nlp(text)


In [ ]:
text = "Apple is looking at buying U.K. startup for $1 billion"
doc = nlp(text)
print(f"{'TEXT': <12} | {'DEP': <10} | {'HEAD TEXT': <12} | {'HEAD POS': <8} | {'CHILDREN'}")
print("-" * 70)

for token in doc:
    children = [child.text for child in token.children]
    print(f"{token.text: <12} | {token.dep_: <10} | {token.head.text: <12} | {token.head.pos_: <8} | {children}")

TEXT         | DEP        | HEAD TEXT    | HEAD POS | CHILDREN
----------------------------------------------------------------------
Apple        | nsubj      | looking      | VERB     | []
is           | aux        | looking      | VERB     | []
looking      | ROOT       | looking      | VERB     | ['Apple', 'is', 'at']
at           | prep       | looking      | VERB     | ['buying']
buying       | pcomp      | at           | ADP      | ['startup', 'for']
U.K.         | compound   | startup      | NOUN     | []
startup      | dobj       | buying       | VERB     | ['U.K.']
for          | prep       | buying       | VERB     | ['billion']
$            | quantmod   | billion      | NUM      | []
1            | compound   | billion      | NUM      | []
billion      | pobj       | for          | ADP      | ['$', '1']


In [8]:
text = "The cat chased the mouse and the dog watched them."
doc = nlp(text)

for token in doc:
    if token.pos_ == "VERB":
        verb = token.text
        subject = ""
        obj = ""

        for child in token.children:
            if child.dep_ == "nsubj":
                subject = child.text
            if child.dep_ == "dobj":
                obj = child.text
        
        for child in token.children:
            if child.dep_ == "conj" and child.pos_ == "VERB":
                conj_verb = child.text
                conj_subj = ""
                conj_obj = ""

                for c in child.children:
                    if c.dep_ == "nsubj":
                        conj_subj = c.text
                    if c.dep_ == "dobj":
                        conj_obj = c.text

                if conj_subj and conj_obj:
                    print(f"Found Triplet: ({conj_subj}, {conj_verb}, {conj_obj})")

        if subject and obj:
            print(f"Found Triplet: ({subject}, {verb}, {obj})")


Found Triplet: (cat, chased, mouse)


In [ ]:
text = "The big, fluffy white cat is sleeping on the warm mat."
doc = nlp(text)

for token in doc:
    if token.pos_ == "NOUN":
        adjectives = []
    
        for child in token.children:
            if child.dep_ == "amod":
                adjectives.append(child.text)
                
        if adjectives:
            print(f"Danh từ '{token.text}' được bổ nghĩa bởi các tính từ: {adjectives}")

Danh từ 'cat' được bổ nghĩa bởi các tính từ: ['big', 'white']
Danh từ 'mat' được bổ nghĩa bởi các tính từ: ['warm']


# Luyen tap

In [ ]:
def find_main_verb(doc):
    """Tìm token là động từ chính (ROOT) của câu."""
    for token in doc:
        if token.dep_ == "ROOT" and token.pos_ == "VERB":
            return token
    return None

text = "The quick brown fox jumps over the lazy dog."
doc = nlp(text)
main_verb = find_main_verb(doc)

if main_verb:
    print(f"Động từ chính (ROOT): {main_verb.text}")
else:
    print("Không tìm thấy động từ chính.")

Động từ chính (ROOT): jumps


In [ ]:
def get_path_to_root(token):
    """Tìm đường đi từ một token lên đến gốc (ROOT)."""
    path = [token]
    while token.head != token:
        token = token.head
        path.append(token)
    return path

text = "Apple is looking at buying U.K. startup for $1 billion"
doc = nlp(text)

startup_token = doc[6]

path = get_path_to_root(startup_token)

path_texts = [token.text for token in path]
path_deps = [token.dep_ for token in path]

print(f"Đường đi từ '{startup_token.text}' đến ROOT:")
print(f"Token: {' -> '.join(path_texts)}") 
print(f"Quan hệ: {' -> '.join(path_deps)}") 

Đường đi từ 'startup' đến ROOT:
Token: startup -> buying -> at -> looking
Quan hệ: dobj -> pcomp -> prep -> ROOT


# Báo cáo Thực hành: Dependency Parsing với spaCy

## 1. Giới thiệu
Phân tích cú pháp phụ thuộc (Dependency Parsing) là kỹ thuật xác định cấu trúc ngữ pháp của câu bằng cách mô hình hóa quan hệ giữa các từ theo cặp head – dependent.

---

## 2. Chuẩn bị môi trường

    pip install -U spacy
    python -m spacy download en_core_web_md

    import spacy
    from spacy import displacy

    nlp = spacy.load("en_core_web_md")

---

## 3. Phân tích và trực quan hóa dependency

    text = "The quick brown fox jumps over the lazy dog."
    doc = nlp(text)
    displacy.serve(doc, style="dep")

Kết quả:
- ROOT: jumps
- nsubj: fox
- prep: over

---

## 4. Trích xuất dependency của từng token

    text = "Apple is looking at buying U.K. startup for $1 billion"
    doc = nlp(text)
    for token in doc:
        children = [child.text for child in token.children]
        print(token.text, token.dep_, token.head.text, children)

Kết luận:
- ROOT: looking
- pcomp: buying
- dobj: startup
- pobj: billion

---

## 5. Trích xuất Subject – Verb – Object

    text = "The cat chased the mouse and the dog watched them."
    doc = nlp(text)
    for token in doc:
        if token.pos_ == "VERB":
            subject = ""
            obj = ""
            for child in token.children:
                if child.dep_ == "nsubj": subject = child.text
                if child.dep_ == "dobj": obj = child.text
            if subject and obj:
                print(subject, token.text, obj)

Kết quả:
    cat chased mouse

---

## 6. Trích xuất tính từ bổ nghĩa danh từ

    text = "The big, fluffy white cat is sleeping on the warm mat."
    doc = nlp(text)
    for token in doc:
        if token.pos_ == "NOUN":
            adjectives = []
            for child in token.children:
                if child.dep_ == "amod": adjectives.append(child.text)
            if adjectives:
                print(token.text, adjectives)

Kết quả:
    cat ['big', 'white']
    mat ['warm']

---

## 7. Tìm động từ chính (ROOT)

    def find_main_verb(doc):
        for token in doc:
            if token.dep_ == "ROOT" and token.pos_ == "VERB":
                return token

    text = "The quick brown fox jumps over the lazy dog."
    doc = nlp(text)
    main_verb = find_main_verb(doc)
    print(main_verb.text)

Kết quả:
    jumps

---

## 8. Đường đi tới ROOT

    def get_path_to_root(token):
        path = [token]
        while token.head != token:
            token = token.head
            path.append(token)
        return path

    text = "Apple is looking at buying U.K. startup for $1 billion"
    doc = nlp(text)
    startup_token = doc[6]
    path = get_path_to_root(startup_token)
    print([t.text for t in path])

Kết quả:
    ['startup', 'buying', 'at', 'looking']

---

## 9. Kết luận

Qua buổi thực hành, tôi đã:
- hiểu dependency tree và ý nghĩa head–dependent
- xác định ROOT
- trích xuất quan hệ S-V-O
- tìm bổ nghĩa của danh từ thông qua amod
- lần theo quan hệ đến ROOT để phân tích cấu trúc câu

Ứng dụng thực tế:
- trích xuất quan hệ (Relation Extraction)
- hệ thống hỏi đáp
- phân tích ngữ nghĩa
- khai thác dữ liệu pháp luật, trợ lý RAG

---
